In [1]:
!pip install ensemble_boxes

     |████████████████████████████████| 3.3 MB 20.2 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 57.4 MB/s eta 0:00:01


In [2]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [20]:
# ensemble csv files
# submission_files = ['./final/submission (5967).csv',
#                    './final/submission (6031).csv',
#                    './final/submission (6192).csv',
#                    './final/submission_fpncarafe50.csv',
#                    './final/submission_latest.csv',
#                    './final/YoloV5x6__Create_Data.csv',
                #    './final/submission(6161).csv'
                #    ] # submission lists
submission_files = ['./Mmdetection/_MyModel/final_output_nms.csv',
                   './Mmdetection/_MyModel/final_output_softnms.csv',
                   './Mmdetection/_MyModel/final_output_nmw.csv',
                   './Mmdetection/_MyModel/final_output_wbf.csv'
                   ]
submission_df = [pd.read_csv(file) for file in submission_files]

In [21]:
image_ids = submission_df[0]['image_id'].tolist()

In [22]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = './dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [24]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
weights = [1,1,1,1] # ensemble weights for model 1 and model 2
iou_thr = 0.6
skip_box_thr = 0.0001 # 원래는 0.01
sigma = 0.1

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        # boxes, scores, labels = nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)
        # boxes, scores, labels = soft_nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
        # boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


In [25]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('/opt/ml/level2-object-detection-level2-cv-18/Mmdetection/_MyModel/final_output_all.csv',index=None)

submission.head()

,PredictionString,image_id
0,7 0.5377773642539978 602.7556762695312 521.100...,test/0000.jpg
1,5 0.47733619809150696 133.16380310058594 0.089...,test/0001.jpg
2,1 0.48052778840065 422.782958984375 268.144744...,test/0002.jpg
3,9 0.4535835087299347 144.19436645507812 261.91...,test/0003.jpg
4,1 0.4446074068546295 204.85708618164062 271.71...,test/0004.jpg
